# Import des librairies

In [1]:
import numpy as np
import pandas as pd
from math import *
from sklearn.impute import KNNImputer


# Import et traitement de la table BUREAU

# I - Import de la table BUREAU

In [2]:
dt_bureau = pd.read_csv("Data_Base/bureau.csv", header = 0, sep = ',')
dt_bureau = dt_bureau[['SK_ID_CURR', 'SK_ID_BUREAU','CREDIT_ACTIVE', 'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM']]

In [3]:
dt_bureau_copie = dt_bureau

In [4]:
dt_bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM
0,215354,5714462,Closed,0,91323.00
1,215354,5714463,Active,0,225000.00
2,215354,5714464,Active,0,464323.50
3,215354,5714465,Active,0,90000.00
4,215354,5714466,Active,0,2700000.00
...,...,...,...,...,...
1716423,259355,5057750,Active,0,11250.00
1716424,100044,5057754,Closed,0,38130.84
1716425,100044,5057762,Closed,0,15570.00
1716426,246829,5057770,Closed,0,36000.00


# II - Analyse des données brutes

## Analyse des valeurs uniques de chaque colonnes

#### Analyse globale

In [5]:
dt_bureau.describe()

,SK_ID_CURR,SK_ID_BUREAU,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM
count,1.716428e+06,1.716428e+06,1.716428e+06,1.716415e+06
mean,2.782149e+05,5.924434e+06,6.410406e-03,3.549946e+05
std,1.029386e+05,5.322657e+05,9.622391e-02,1.149811e+06
min,1.000010e+05,5.000000e+06,0.000000e+00,0.000000e+00
25%,1.888668e+05,5.463954e+06,0.000000e+00,5.130000e+04
50%,2.780550e+05,5.926304e+06,0.000000e+00,1.255185e+05
75%,3.674260e+05,6.385681e+06,0.000000e+00,3.150000e+05
max,4.562550e+05,6.843457e+06,9.000000e+00,5.850000e+08


In [6]:
dt_columns = dt_bureau.columns.values.tolist()

for i in dt_columns :
    print(f'colonne {i} : {dt_bureau[i].unique()}\n')

colonne SK_ID_CURR : [215354 162297 402440 ... 448157 345866 235871]

colonne SK_ID_BUREAU : [5714462 5714463 5714464 ... 5057762 5057770 5057778]

colonne CREDIT_ACTIVE : ['Closed' 'Active' 'Sold' 'Bad debt']

colonne CNT_CREDIT_PROLONG : [0 2 1 4 3 5 9 8 6 7]

colonne AMT_CREDIT_SUM : [ 91323.   225000.   464323.5  ...  77861.43 112204.35 108765.72]



#### Analyse des valeurs diverses

In [7]:
columns_to_inspect = ['CNT_CREDIT_PROLONG','AMT_CREDIT_SUM']

print(f'Nb incompatible data per column :\n')

#for i in dt_columns_inspect :
  
analyse = dt_bureau[columns_to_inspect].describe()

for i in columns_to_inspect :
    
    print(f"{i} :")
    print(f"\t -> MIN : {analyse[i]['min']}")
    print(f"\t -> MAX : {analyse[i]['max']}")
    print(f"\n-------------------------\n")

Nb incompatible data per column :

CNT_CREDIT_PROLONG :
	 -> MIN : 0.0
	 -> MAX : 9.0

-------------------------

AMT_CREDIT_SUM :
	 -> MIN : 0.0
	 -> MAX : 585000000.0

-------------------------



#### Analyse des NaN

In [8]:
dt_columns = dt_bureau.columns

incomplet_data = 0

column_with_nan = []
tab_count_nan = []

for i in dt_columns :
    verif = dt_bureau[i].isna().sum()
    if verif != 0 :
        column_with_nan.append(i)
        tab_count_nan.append(dt_bureau[i].isna().sum())
        
    incomplet_data += dt_bureau[i].isna().sum()

nan_analysis = pd.DataFrame({'Feature' : column_with_nan ,  'Nb_NaN' : tab_count_nan})

print(f"{len(column_with_nan)} colonnes contient au moins un 'NaN' :")

nan_analysis

1 colonnes contient au moins un 'NaN' :


,Feature,Nb_NaN
0,AMT_CREDIT_SUM,13


In [9]:
total_info = len(dt_bureau['SK_ID_CURR']) * len(dt_columns)

print(f"Total Data = {total_info}")
print(f"Missing Data = {incomplet_data}")
print(f"Missing Data Rate = {round( (incomplet_data / total_info),2) * 100} %")

Total Data = 8582140
Missing Data = 13
Missing Data Rate = 0.0 %


### Résultats de l'analyse :

- La colonne CREDIT_ACTIVE doit être mise à plat


- Les NaN sont à traiter


- Créations de nouvelles colonnes :
    * Total de crédits précédent le crédit actuel
    * Nombre de crédits terminés
    * Nombre de crédits en cours
    * Somme total des crédits terminés
    * Somme total des crédits en cours
    * Derière date du dernier crédit accordés
    * Somme du dernier crédit
    * Etat du dernier crédit

# III - Nettoyage des données

### Récupération des colonnes à reformer

In [10]:
# récupération des colonnes à reformer
colonne_reform = []

columnes_with_str = dt_bureau.columns

for i in columnes_with_str :
    cran = 1
    while cran != 0 :
        if dt_bureau[i][0] != np.nan :
            cran = 0
            if type(dt_bureau[i][0]) == str :
                colonne_reform.append(i)

colonne_reform

['CREDIT_ACTIVE']

### Création d'un dictionnaire

In [11]:
longest_list = 0


# récupération du plus grand nombre de valeurs uniques présent dans une colonne 
for i in colonne_reform :
    if len(dt_bureau[i].unique()) > longest_list :
        longest_list = len(dt_bureau[i].unique())


# création d'un tableau de sauvegarde des colonnes contenant des valeurs uniques de type "chaine de caractères"      
values_save = [[''] * (longest_list+1) for x in range(len(colonne_reform))]

# remplissage du tableau de sauvegarde
for i in range (len(colonne_reform)) :
    
    values_save[i][0] = colonne_reform[i]
    
    valeurs_uniques = dt_bureau[colonne_reform[i]].unique()
    
    j = 1
    diff = 1
        
    for j in range (len(valeurs_uniques)) :
   
        if type(valeurs_uniques[j]) == str :
            values_save[i][j+diff] = valeurs_uniques[j]
        else :
            diff -= 1
        

values_save

[['CREDIT_ACTIVE', 'Closed', 'Active', 'Sold', 'Bad debt']]

##### Transformation des chaines de caractère en équivalent de type integer

In [12]:
for i in range (len(values_save)) :
    for j in range (len(values_save[i])) :
        dt_bureau[values_save[i][0]] = dt_bureau[values_save[i][0]].replace(values_save[i][j], j)

## Traitement des NaN

### Récupération des featurs avec NaN

REMARQUE : on sélectionne toutes les données parce que nous n'avons pas beaucoup de colonnes et pas beaucoup de NaN, ça ne sert à rien de perdre du temps et de la ressource de calcul à effectuer des étapes non essentielles

In [13]:
X_with_NaN = dt_bureau.drop(['SK_ID_CURR', 'SK_ID_BUREAU'], axis = 1)

### Application du KNNImputer

In [14]:
imputer = KNNImputer(n_neighbors = 5)
        
X_without_nan = imputer.fit_transform(X_with_NaN)
        
df_result = pd.DataFrame(X_without_nan,columns=X_with_NaN.columns)

### Récupération de la colonne concernée par la réctification des NaN 

In [15]:
columns_with_NA = []
for i in nan_analysis['Feature'] :
    columns_with_NA.append(i)

df_result = df_result[columns_with_NA]

### Unification de la donnée Finale

#### Retrait des anciennes colonnes avec NaN

In [16]:
columnt_to_delet = df_result.columns.values.tolist()
dt_bureau = dt_bureau.drop(columnt_to_delet, axis = 1)

#### Construction d'une colonnes de jointure dans la table des résultats de traitement des NaN

In [17]:
index_id_bureau = []

for i in dt_bureau['SK_ID_BUREAU']:
    index_id_bureau.append(i)
    
df_result['SK_ID_BUREAU'] = index_id_bureau

#### Reconstruction du DF sans NaN

In [18]:
dt_bureau = pd.merge(dt_bureau, df_result, on = 'SK_ID_BUREAU')

In [19]:
for i in range (len(values_save)) :
    for j in range (len(values_save[i])) :
        dt_bureau[values_save[i][0]] = dt_bureau[values_save[i][0]].replace(j,values_save[i][j])

In [20]:
dt_bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM
0,215354,5714462,Closed,0,91323.00
1,215354,5714463,Active,0,225000.00
2,215354,5714464,Active,0,464323.50
3,215354,5714465,Active,0,90000.00
4,215354,5714466,Active,0,2700000.00
...,...,...,...,...,...
1716423,259355,5057750,Active,0,11250.00
1716424,100044,5057754,Closed,0,38130.84
1716425,100044,5057762,Closed,0,15570.00
1716426,246829,5057770,Closed,0,36000.00


## Création de nouvelles colonnes

### Totale de la somme crédité avant l'application  ==> TOTAL_AMT_CREDIT

In [22]:
credit_total = dt_bureau[['SK_ID_CURR', 'AMT_CREDIT_SUM']]
df_somme_credit = credit_total.groupby(credit_total['SK_ID_CURR']).agg({"AMT_CREDIT_SUM": "sum",})

### Création du Data Frame final

#### Listing de tous les id uniques de crédits

In [23]:
dt_bureau = dt_bureau['SK_ID_CURR']
SK_ID_CURR = dt_bureau.unique()
final_df = pd.DataFrame({'SK_ID_CURR' : SK_ID_CURR})

#### Concaténation des indicateurs précédement calculés

In [24]:
final_df = pd.merge(final_df, df_somme_credit, on = 'SK_ID_CURR') # => TOTAL_AMT_CREDIT

#### Choix nom des colonnes

In [25]:
final_columns = ['SK_ID_CURR', 'TOTAL_AMT_CREDIT']

#### Calcul de la solvabilité du créditeur (pourcentage de somme payé sur l'ensemble des sommes empreintées

In [26]:
final_df.columns = final_columns

## Export du Data Frame final

In [27]:
final_df

,SK_ID_CURR,TOTAL_AMT_CREDIT
0,215354,5973945.30
1,162297,8230386.15
2,402440,89910.00
3,238881,1285239.06
4,222183,7158960.00
...,...,...
305806,207190,450000.00
305807,324956,19800.00
305808,448157,1800000.00
305809,345866,175054.50


In [28]:
final_df.to_csv('Data_Base_Enhanced/bureau_enhanced.csv', index = False)